In [ ]:
# Set up colab instance
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Make sure clone at root
!pip3 install pydicom
!git clone https://github.com/thomasp05/gif-705-projet

In [ ]:
import os
os.chdir('gif-705-projet')

In [ ]:
!git pull origin features/metrics
!git checkout features/metrics

In [ ]:
import time

import torch

from dataset import *
from models import *
from metrics import *

import models_parts

torch.manual_seed(111)

In [ ]:
class Downsample:
  def __init__(self):
    self.pool = nn.AvgPool2d(2)
    
  def __call__(self, x, target):
    x = self.pool(x.unsqueeze(0)).squeeze(0)
    target = self.pool(target)
    return x, target

In [ ]:
dataset = dcm_dataset('../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/data',
                      transforms=Downsample())
print("Found {} images".format(len(dataset.img_files)))

train_set, test_set = train_test_split(dataset)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=4, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=4, num_workers=2)

In [ ]:
model = UNet(1).to("cuda:0")

In [ ]:
run_test(model, test_loader, [IoU(), Confusion_matrix()], "cuda:0")